# **ĐỒ ÁN THỰC HÀNH CUỐI KỲ**

## Môn: **Toán ứng dụng và thống kê**
## Lớp: **21_2**
## Họ tên: **Trương Thành Nhân** - MSSV **21120105**

## **ĐỀ 2** Bài toán data fitting
- Sử dụng bài toán data fitting trong **xây dựng mô hình đánh giá lương nhân viên** từ các yếu tố tác động theo dữ liệu được cung cấp.
- Sử dụng bài toán data fitting trong việc **xây dựng mô hình đánh giá giá nhà** từ các yếu tố tác động từ dữ liệu được cung cấp.

## **1. THƯ VIỆN VÀ CÁC HÀM HỖ TRỢ**

### **1.1 THƯ VIỆN HỖ TRỢ**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

### **1.2 CÁC HÀM HỖ TRỢ**

#### 1.2.1 HÀM <code>inverse(Matrix)</code>
**Hàm tìm ma trận nghịch đảo của ma trận <code>Matrix</code> ban đầu**

In [2]:
def inverse(Matrix):
    n = len(Matrix)
    result = [[0 for i in range(n)] for j in range(n)]
    In = [[0 for i in range(n)] for j in range(n)]
    for i in range(n):
        In[i][i] = 1
    B = [Matrix[i] + In[i] for i in range(n)]
    for i in range(n):
        div = B[i][i]
        for j in range(2*n):
            B[i][j] /= div
        for j in range(n):
            if i != j:
                sub = B[j][i]
                for k in range(2*n):
                    B[j][k] -= sub * B[i][k]
    for i in range(n):
        result[i] = B[i][n:]
    return result

### 1.2.2 HÀM <code>transpose(Matrix)<code>
**Hàm tìm ma trận chuyển vị của ma trận <code>Matrix</code> ban đầu**

In [3]:
def transpose(Matrix):
    if len(Matrix) == 0 and len(Matrix[0]) == 0:
        return []
    A = []
    for i in range(len(Matrix[0])):
        A.append([])
        for j in range(len(Matrix)):
            A[i].append(Matrix[j][i])
    return A

### 1.2.3 HÀM <code>multiplyMatrix(A, B)<code>
**Hàm nhân hai ma trận A và B**

In [4]:
def multiplyMatrix(A, B): 
    if len(A[0]) != len(B):
        return "Không thể nhân hai ma trận này"
    result = [[0 for i in range(len(B[0]))] for j in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

### 1.2.4 HÀM <code>addColumn(Matrix)<code>
**Hàm bổ sung thêm cột số 1 vào ma trận <code>Matrix</code>**

In [5]:
def addColumn(Matrix):
    # Tạo ma trận mới với số cột lớn hơn 1
    newMatrix = [[None]*(len(Matrix[0])+1) for _ in range(len(Matrix))]
    # Chép phần tử từ ma trận ban đầu vào ma trận mới
    for i in range(len(Matrix)):
        newMatrix[i][1:] = Matrix[i][:]
    # Chèn cột mới vào ma trận mới
    for i in range(len(newMatrix)):
        newMatrix[i][0] = 1
    return newMatrix

### 1.2.5 HÀM <code>linearRegressionModel(A, Y)<code>
**Hàm tính vector chứa các hệ số của mô hình hồi quy dựa vào <code>ma trận các yếu tố tác động A</code> và <code>ma trận cột Y chứa các giá trị ban đầu</code>**

In [6]:
def linearRegressionModel(A, Y):
    _A = transpose(A)                   # A^T
    T1 = inverse(multiplyMatrix(_A, A)) # (A^T.A)^-1
    T2 = multiplyMatrix(_A, Y)          # A^T.Y
    y = multiplyMatrix(T1,T2)           # (A^T.A)^-1.A^T.Y
    for i in range(len(y)):             # Làm tròn kết quả đến chữ số thập phân thứ 7
        y[i][0] = round(y[i][0],7)
    return y

### 1.2.6 HÀM <code>findResidual(A, y, Y)<code>
**Hàm tính chuẩn vector phần dư <code>||r||</code>**

In [7]:
def findResidual(A, Y, y):
    R = [[None]*(len(Y[0])) for _ in range(len(Y))]
    for i in range(len(R)):
        tmp = 0
        for j in range(len(y)):
            tmp += A[i][j]*y[j][0]
        R[i][0] = tmp - Y[i][0]      # Tìm các giá trị ri = yi^ - yi
    result = 0
    for i in range(len(R)):
        result += R[i][0]*R[i][0]
    return round(result**(1/2),7)    # Làm tròn kết quả đến chữ số thập phân thứ 7

### 1.2.7 HÀM <code>printLinearRegressionModel(y, check)<code>
**Hàm in ra mô hình hồi quy tìm được, với <code>check</code> là một số nguyên ($1\leq check \leq 4$):**
- check = 1: mô hình $y = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_{3}x_{3} $
- check = 2: mô hình $\ln{y} = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_{3}x_{3}$
- check = 3: mô hình $\ln{y} = \theta_0 + \theta_1 ln{x_1} + \theta_2 ln{x_2} + \theta_3 ln{x_3} + \theta_4 ln{x_4}$
- check = 4: mô hình $y = \theta_0 + \theta_1x_1 + \theta_2x_2^{2} + \theta_{3}x_{3}^{3}$

In [8]:
def printLinearRegressionModel(y, check):
    if (check == 1 or check == 4):
        model = "y = " + str(y[0][0])
    if (check == 2 or check == 3):
        model = "lny = " + str(y[0][0])
    for i in range(len(y)-1):
        if (y[i+1][0] == 0):
                model += ""
        elif (check == 1 or check == 2 or check == 4):
            if (y[i+1][0] > 0):
                model += " + " + str(y[i+1][0]) + "*" + str('x' + str(i + 1))
            elif (y[i+1][0] < 0):
                model += " - " + str(abs(y[i+1][0])) + "*" + str('x' + str(i + 1))
            if (check == 4):
                model += '^'+str(i+1)
        elif (check == 3):
            if (y[i+1][0] > 0):
                model += " + " + str(y[i+1][0]) + "*" + str('lnx' + str(i + 1))
            elif (y[i+1][0] < 0):
                model += " - " + str(abs(y[i+1][0])) + "*" + str('lnx' + str(i + 1))
    if (check == 2 or check == 3):
        model += "\nhay\n\ty = e^(" + model.lstrip("lny = ") + ')'
    return model        

### 1.2.8 HÀM <code>printResult(result, A, y, check)</code>

**Hàm in ra các <code>hệ số</code> của mô hình hồi quy tuyến tính, <code>mô hình hồi quy tuyến tính</code> và <code>chuẩn của vector phần dư</code> của mô hình tìm được**

In [9]:
def printResult(result, A, y, check):
    print("v* =", result)
    print("\nMô hình hồi thu được là: \n\t", printLinearRegressionModel(result, check))
    print("\nChuẩn vector phần dư (residual): ||r1||=", findResidual(A, y, result))

### **2. MÔ HÌNH ĐÁNH GIÁ LƯƠNG NHÂN VIÊN**
#### Ta sẽ xét một vài mô hình, sau đó dựa trên chuẩn vector phần dư ||r|| để tìm ra mô hình tối ưu hơn so với những mô hình còn lại

In [10]:
# dùng thư viện pandas cho việc đọc dữ liệu
df = pd.read_excel('cps4_small.xlsx')
df

,wage,educ,exper,hrswk
0,18.70,16,39,37
1,11.50,12,16,62
2,15.04,16,13,40
3,25.95,14,11,40
4,24.03,12,51,40
...,...,...,...,...
995,16.83,16,27,40
996,28.85,13,7,40
997,11.25,14,5,40
998,7.50,8,36,40


In [11]:
data = np.array(df) # Sử dụng thư viện numpy chuyển toàn bộ các giá trị trong data sang array
a = data[:,-3:]     # Lấy các cột chứa các yếu tố tác động đến lương nhân viên
y = data[:,:-3]     # lấy cột lương của các nhân viên

#### **2.1 Sử dụng mô hình tuyến tính $y = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_{3}x_{3} $**

In [12]:
A = addColumn(a)                      # Thêm một cột chứa các số 1 vào ma trận a
result = linearRegressionModel(A,y)   # Hàm tìm các hệ số hồi quy
printResult(result, A, y, check = 1)  # In các kết quả tìm được

v* = [[-16.4432252], [2.0119945], [0.1437088], [0.1373133]]

Mô hình hồi thu được là: 
	 y = -16.4432252 + 2.0119945*x1 + 0.1437088*x2 + 0.1373133*x3

Chuẩn vector phần dư (residual): ||r1||= 361.0899821


#### **2.2 Sử dụng mô hình log - tuyến tính $\ln{y} = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_{3}x_{3}$**

In [13]:
# Tính giá trị ln của các phần tử trong cột lương
newY = [[None]*(len(y[0])) for _ in range(len(y))]
for i in range(len(y)):
    newY[i][0] = math.log(y[i])

A = addColumn(a)                          # Bổ sung cột số 1 vào ma trận a  
result = linearRegressionModel(A, newY)   # Tìm các hệ số của mô hình hồi quy
printResult(result, A, newY, check = 2)   # In các kết quả tìm được

v* = [[1.1005398], [0.0903056], [0.0057759], [0.0089411]]

Mô hình hồi thu được là: 
	 lny = 1.1005398 + 0.0903056*x1 + 0.0057759*x2 + 0.0089411*x3
hay
	y = e^(1.1005398 + 0.0903056*x1 + 0.0057759*x2 + 0.0089411*x3)

Chuẩn vector phần dư (residual): ||r1||= 16.2111707


#### **2.3 Sử dụng mô hình đa thức $y = \theta_0 + \theta_1x_1 + \theta_2x_2^{2} + \theta_{3}x_{3}^{3}$**

In [14]:
# Ứng với mỗi cột trong cùng hàng thứ i là x_i với hệ số mũ tương ứng giống như mô hình đã cho ban đầu
newA = [[None]*(len(a[0])) for _ in range(len(a))]
for i in range(len(newA)):
    for j in range(len(newA[0])):
        newA[i][j] = a[i][j]**(j+1)

A = addColumn(newA)                    # Bổ sung cột số 1 vào ma trận newA
result = linearRegressionModel(A, y)   # Tìm các hệ số của mô hình hồi quy
printResult(result, A, y, check = 4)   # In các kết quả tìm được

v* = [[-9.45003], [2.0132602], [0.001659], [1.11e-05]]

Mô hình hồi thu được là: 
	 y = -9.45003 + 2.0132602*x1^1 + 0.001659*x2^2 + 1.11e-05*x3^3

Chuẩn vector phần dư (residual): ||r1||= 365.9717226


### **Nhận xét:**
Ta thấy $||r_3|| > ||r_1|| >> ||r_2|| (365.9717226 > 361.0899821 >> 16.2111707)$ nên mô hình ở **2.2 log - tuyến tính** sẽ tối ưu hơn.

Vậy ta lựa chọn **mô hình đánh giá lương nhân viên** là: $$\ln{y} = 1.1005398 + 0.0903056x_1 + 0.0057759x_2 + 0.0089411x_3$$
hay $$y = e^{1.1005398 + 0.0903056x_1 + 0.0057759x_2 + 0.0089411x_3}$$
Chuẩn vector phần dư (residual): $||r||= 16.2111707$

### **3. MÔ HÌNH ĐÁNH GIÁ GIÁ NHÀ**
#### Ta sẽ xét một vài mô hình, sau đó dựa trên chuẩn vector phần dư ||r|| để tìm ra mô hình tối ưu hơn so với những mô hình còn lại

In [15]:
# dùng thư viện pandas cho việc đọc dữ liệu
house = pd.read_excel('br2.xlsx')
house

,price,sqft,Bedrooms,Baths,Age
0,66500,741,1,1,18
1,66000,741,1,1,18
2,68500,790,1,1,18
3,102000,2783,2,2,18
4,54000,1165,2,1,35
...,...,...,...,...,...
1075,122570,2853,5,3,25
1076,185000,4599,5,3,13
1077,1280000,7086,5,3,13
1078,123808,3148,5,2,25


In [16]:
data = np.array(house)   # Sử dụng thư viện numpy chuyển toàn bộ các giá trị trong data sang array
m = data[:,-4:]          # Lấy các cột chứa các yếu tố tác động đến giá nhà
b = data[:,:-4]          # lấy cột giá nhà

#### **3.1 Sử dụng mô hình tuyến tính $y = \theta_0 + \theta_1x_1 + \theta_2x_2 + ... + \theta_{n}x_{n}$**

In [17]:
M = addColumn(m)                       # Thêm một cột chứa các số 1 vào ma trận m
result = linearRegressionModel(M, b)   # Tìm các hệ số của mô hình hồi quy
printResult(result, M, b, check = 1)   # In các kết quả tìm được

v* = [[-26733.1238772], [87.5006884], [-29192.6536254], [41420.7677794], [-553.4032903]]

Mô hình hồi thu được là: 
	 y = -26733.1238772 + 87.5006884*x1 - 29192.6536254*x2 + 41420.7677794*x3 - 553.4032903*x4

Chuẩn vector phần dư (residual): ||r1||= 2493863.7509459


#### **3.2 Sử dụng mô hình log - log $\ln{y} = \theta_0 + \theta_1 ln{x_1} + \theta_2 ln{x_2} + \theta_3 ln{x_3} + \theta_4 ln{x_4}$**

In [18]:
# Tính giá trị ln của các phần tử trong ma trận cột giá nhà b
newB = [[None]*(len(b[0])) for _ in range(len(b))]
for i in range(len(b)):
    newB[i][0] = math.log(b[i])

# Tính giá trị ln của các phần tử của các cột tính chất trong ma trận m
newm = [[None]*(len(m[0])) for _ in range(len(m))]
for i in range(len(m)):
    for j in range(len(m[0])):
        newm[i][j] = math.log(m[i][j])
        
M = addColumn(newm)                       # Bổ sung cột số 1 vào ma trận newm
result = linearRegressionModel(M, newB)   # Tìm các hệ số của mô hình hồi quy
printResult(result, M, newB, check = 3)   # In các kết quả tìm được

v* = [[5.4879381], [0.8514173], [-0.2802541], [0.3992267], [-0.0669347]]

Mô hình hồi thu được là: 
	 lny = 5.4879381 + 0.8514173*lnx1 - 0.2802541*lnx2 + 0.3992267*lnx3 - 0.0669347*lnx4
hay
	y = e^(5.4879381 + 0.8514173*lnx1 - 0.2802541*lnx2 + 0.3992267*lnx3 - 0.0669347*lnx4)

Chuẩn vector phần dư (residual): ||r1||= 9.683687


#### **3.3 Sử dụng mô hình log - tuyến tính $\ln{y} = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_{3}x_{3} + \theta_{4}x_{4}$**

In [19]:
# Tính giá trị ln của các phần tử trong ma trận cột giá nhà b
newB = [[None]*(len(b[0])) for _ in range(len(b))]
for i in range(len(b)):
    newB[i][0] = math.log(b[i])

M = addColumn(m)                          # Bổ sung cột số 1 vào ma trận a
result = linearRegressionModel(M, newB)   # Tìm các hệ số của mô hình hồi quy
printResult(result, M, newB, check = 2)   # In các kết quả tìm được

v* = [[10.9189639], [0.0003308], [-0.0589539], [0.2145707], [-0.0066016]]

Mô hình hồi thu được là: 
	 lny = 10.9189639 + 0.0003308*x1 - 0.0589539*x2 + 0.2145707*x3 - 0.0066016*x4
hay
	y = e^(10.9189639 + 0.0003308*x1 - 0.0589539*x2 + 0.2145707*x3 - 0.0066016*x4)

Chuẩn vector phần dư (residual): ||r1||= 9.1239914


### **Nhận xét:**
Ta thấy $||r_1|| >> ||r_2|| > ||r_3||  (2493863.7509459 >> 9.683687 > 9.1239914)$ nên mô hình ở **3.2 log - tuyến tính** sẽ tối ưu hơn.

Vậy ta lựa chọn **mô hình đánh giá giá nhà** là: $$ \ln{y} = 10.9189639 + 0.0003308x_1 - 0.0589539x_2 + 0.2145707x_3 - 0.0066016x_4$$
hay $$y = e^{10.9189639 + 0.0003308x_1 - 0.0589539x_2 + 0.2145707x_3 - 0.0066016x_4}$$
Chuẩn vector phần dư (residual): $||r||= 9.1239914$